In [1]:
import os
import sys

import pandas as pd
import numpy as np
import datetime

from django.db import IntegrityError

from tv import models as tv_models


# Change DataFrame dtypes to match Database models
def model_for_db(csv):
    """
    @parameters:
    - csv: Electronic Guide filename in comma separated values (.csv)

    @returns:
    - Dataframe with items formatted for bmat tv/av programs
    """

    # Load csv into Dataframe
    df = pd.read_csv(csv)

    # Fill n/a with readable string "n/a"
    df = df.fillna('n/a')
    df['program_year'] = df['program_year'].astype(str)
    df['start_time'] = (df['start_date'].astype(str) + ' ' + df['start_time']).map(
        lambda x: datetime.datetime.strptime(x, '%Y%m%d %H:%M')
    )
    df['duration_in_seconds'] = pd.to_timedelta(df['duration_in_seconds'], unit='seconds')
    df['end_time'] = df['start_time'] + df['duration_in_seconds']
    df = df.drop(['start_date', 'duration_in_seconds'], axis=1)
    return df


# Get or create Program with channel obj and row data
def create_program(row, channel):
    if row.program_year != 'n/a':
        year = row.program_year[0:4]
    else:
        year = row.program_year
    program, created =tv_models.Program.objects.get_or_create(uid=row.program_id, channel=channel,
                                                            year=year,
                                                            local_title=row.program_local_title,
                                                            original_title=row.program_original_title
                                                            )
    if created:
        program.save()
        print("Program {} with id {} added to DB.".format(program.local_title,
                                                          program.uid,
                                                          ))
    return program


def create_channel(row):
    channel, created = tv_models.Channel.objects.get_or_create(uid=row.channel_id,
                                                     name=row.channel_name,
                                                     country_code=row.channel_country
                                                     )
    if created:
        channel.save()
        print('Channel {} from {} with id {} added to DB.'.format(channel.name,
                                                                  channel.country_code,
                                                                  channel.id
                                                                  ))
    return channel


def create_showtime(row, program):
    showtime, created = tv_models.Showtime.objects.get_or_create(program=program,
                                                              start_time=row.start_time,
                                                              end_time=row.end_time
                                                              )
    if created:
        showtime.save()
        print('Showtime starting on {} added to '.format(row.start_time)+program.local_title)
    return showtime


# Create Showtimes from DataFrame
def add_showtimes_from_dataframe(df, filename):
    for index, row in df.iterrows():
        channel = create_channel(row)
        program = create_program(row, channel)
        create_showtime(row, program)

    print("Finished adding programs from {}".format(filename))


In [2]:
! ls .
'../../data/epg.csv'

Programs API.ipynb          __init__.py
Upload EPG from csv.ipynb   upload_from_csv.py
Vericast-reports-tool.ipynb


'../../data/epg.csv'

In [3]:
df = model_for_db('../../data/epg.csv')

In [4]:
add_showtimes_from_dataframe(df, 'epg.csv')

Showtime starting on 2018-01-25 14:30:00 added to La sexta meteo
Showtime starting on 2018-01-25 14:45:00 added to Urgences
Showtime starting on 2018-01-25 14:45:00 added to Zapeando
Showtime starting on 2018-01-25 15:00:00 added to Fair City
Showtime starting on 2018-01-25 15:30:00 added to The Best of Today With Maura and Daithi
Program Urgences with id 13170398 added to DB.
Showtime starting on 2018-01-25 15:35:00 added to Urgences
Program Más vale tarde with id 328017822 added to DB.
Showtime starting on 2018-01-25 16:15:00 added to Más vale tarde
Showtime starting on 2018-01-25 16:30:00 added to Grey's Anatomy
Showtime starting on 2018-01-25 17:20:00 added to Grey's Anatomy
Showtime starting on 2018-01-25 17:40:00 added to Nuacht RTÉ
Showtime starting on 2018-01-25 18:00:00 added to The Angelus
Showtime starting on 2018-01-25 18:01:00 added to RTÉ News: Six One
Showtime starting on 2018-01-25 18:05:00 added to Grey's Anatomy
Program Grey's Anatomy with id 17640623 added to DB.
Sho

Showtime starting on 2018-01-26 20:00:00 added to Les innocents
Showtime starting on 2018-01-26 20:05:00 added to La Sexta deportes
Program #laSexta Columna with id 5731404502 added to DB.
Showtime starting on 2018-01-26 20:30:00 added to #laSexta Columna
Program Francis Brennan's Grand Vietnamese Tour with id 5085731261 added to DB.
Showtime starting on 2018-01-26 20:30:00 added to Francis Brennan's Grand Vietnamese Tour
Program Les innocents with id 5691352991 added to DB.
Showtime starting on 2018-01-26 20:50:00 added to Les innocents
Showtime starting on 2018-01-26 21:00:00 added to RTÉ News: Nine O'Clock
Showtime starting on 2018-01-26 21:30:00 added to Equipo de investigación
Program The Late Late Show with id 5759512361 added to DB.
Showtime starting on 2018-01-26 21:35:00 added to The Late Late Show
Showtime starting on 2018-01-26 21:55:00 added to Section de recherches
Showtime starting on 2018-01-26 23:05:00 added to Programmes de nuit
Program Equipo de investigación with id 

Program NCIS with id 1882225288 added to DB.
Showtime starting on 2018-01-28 02:05:00 added to NCIS
Showtime starting on 2018-01-28 02:30:00 added to Canal Ruleta
Program Our Girl with id 1233324908 added to DB.
Showtime starting on 2018-01-28 02:50:00 added to Our Girl
Showtime starting on 2018-01-28 03:00:00 added to Minutos musicales
Program Waking the Dead with id 8727117 added to DB.
Showtime starting on 2018-01-28 03:45:00 added to Waking the Dead
Showtime starting on 2018-01-28 04:35:00 added to Euronews
Showtime starting on 2018-01-28 05:00:00 added to Minutos musicales
Program Historias con denominación de origen with id 1427440280 added to DB.
Showtime starting on 2018-01-28 05:15:00 added to Historias con denominación de origen
Showtime starting on 2018-01-28 05:20:00 added to Petits secrets entre voisins
Program Bestial with id 5078637288 added to DB.
Showtime starting on 2018-01-28 05:45:00 added to Bestial
Showtime starting on 2018-01-28 05:50:00 added to Petits secrets e

Program Al Rojo Vivo with id 5740316039 added to DB.
Showtime starting on 2018-01-29 10:00:00 added to Al Rojo Vivo
Program The Doctors with id 4566022240 added to DB.
Showtime starting on 2018-01-29 10:10:00 added to The Doctors
Showtime starting on 2018-01-29 10:15:00 added to Au nom de la vérité
Showtime starting on 2018-01-29 10:50:00 added to Au nom de la vérité
Program Dr. Phil with id 4502409309 added to DB.
Showtime starting on 2018-01-29 11:05:00 added to Dr. Phil
Showtime starting on 2018-01-29 11:15:00 added to Au nom de la vérité
Showtime starting on 2018-01-29 11:50:00 added to Au nom de la vérité
Program Shortland Street with id 5741755342 added to DB.
Showtime starting on 2018-01-29 12:00:00 added to Shortland Street
Program Au nom de la vérité with id 5726138304 added to DB.
Showtime starting on 2018-01-29 12:20:00 added to Au nom de la vérité
Program The Shelbourne with id 1992647441 added to DB.
Showtime starting on 2018-01-29 12:30:00 added to The Shelbourne
Showtime

Showtime starting on 2018-01-30 13:55:00 added to La Sexta Noticias: Jugones
Program Neighbours with id 5728773120 added to DB.
Showtime starting on 2018-01-30 14:00:00 added to Neighbours
Showtime starting on 2018-01-30 14:20:00 added to Jugones: El análisis
Showtime starting on 2018-01-30 14:25:00 added to EastEnders
Showtime starting on 2018-01-30 14:30:00 added to La sexta meteo
Program Zapeando with id 5748722958 added to DB.
Showtime starting on 2018-01-30 14:45:00 added to Zapeando
Showtime starting on 2018-01-30 14:45:00 added to Urgences
Program Grandma's Boy with id 1447746547 added to DB.
Showtime starting on 2018-01-30 15:00:00 added to Grandma's Boy
Showtime starting on 2018-01-30 15:30:00 added to The Best of Today With Maura and Daithi
Program Urgences with id 12945234 added to DB.
Showtime starting on 2018-01-30 15:35:00 added to Urgences
Program Más vale tarde with id 5748722952 added to DB.
Showtime starting on 2018-01-30 16:15:00 added to Más vale tarde
Showtime star

...

Showtime starting on 2018-02-23 11:45:00 added to Shortland Street
Program Au nom de la vérité with id 5809389005 added to DB.
Showtime starting on 2018-02-23 12:10:00 added to Au nom de la vérité
Showtime starting on 2018-02-23 12:15:00 added to The Zoo
Showtime starting on 2018-02-23 12:45:00 added to Telly Bingo
Showtime starting on 2018-02-23 12:50:00 added to Petits plats en équilibre
Showtime starting on 2018-02-23 12:55:00 added to Urgences
Showtime starting on 2018-02-23 13:00:00 added to RTÉ News: One O'Clock
Showtime starting on 2018-02-23 13:00:00 added to laSexta Noticias 1ª Edición
Program Home And Away with id 5813225182 added to DB.
Showtime starting on 2018-02-23 13:30:00 added to Home And Away
Showtime starting on 2018-02-23 13:40:00 added to Urgences
Showtime starting on 2018-02-23 13:55:00 added to La Sexta Noticias: Jugones
Program Neighbours with id 5813225234 added to DB.
Showtime starting on 2018-02-23 14:00:00 added to Neighbours
Showtime starting on 2018-02-23 

Showtime starting on 2018-02-24 15:45:00 added to Room to Improve
Program Hydra with id 9393497 added to DB.
Showtime starting on 2018-02-24 16:10:00 added to Hydra
Showtime starting on 2018-02-24 16:45:00 added to Nuacht RTÉ
Showtime starting on 2018-02-24 16:50:00 added to Alice Nevers : le juge est une femme
Showtime starting on 2018-02-24 16:55:00 added to Operation Transformation
Showtime starting on 2018-02-24 17:50:00 added to Alice Nevers : le juge est une femme
Program El todopoderoso Thor with id 1293841647 added to DB.
Showtime starting on 2018-02-24 17:55:00 added to El todopoderoso Thor
Showtime starting on 2018-02-24 18:00:00 added to The Angelus
Showtime starting on 2018-02-24 18:01:00 added to RTÉ News: Six One
Program Mirror Mirror with id 18167392 added to DB.
Showtime starting on 2018-02-24 18:35:00 added to Mirror Mirror
Program Alice Nevers : le juge est une femme with id 12031235 added to DB.
Showtime starting on 2018-02-24 18:45:00 added to Alice Nevers : le juge

Showtime starting on 2018-02-25 22:45:00 added to Expediente Marlasca: historias de malos
Showtime starting on 2018-02-25 23:10:00 added to The Week in Politics
Showtime starting on 2018-02-26 00:00:00 added to À communiquer
Showtime starting on 2018-02-26 00:00:00 added to RTE 1 Programming
Showtime starting on 2018-02-26 00:00:00 added to Será anunciado
Showtime starting on 2018-02-26 00:15:00 added to The Ray D'Arcy Show
Showtime starting on 2018-02-26 00:35:00 added to Programmes de nuit
Showtime starting on 2018-02-26 01:30:00 added to Comprando en casa
Program Call the Midwife with id 892926479 added to DB.
Showtime starting on 2018-02-26 01:40:00 added to Call the Midwife
Showtime starting on 2018-02-26 02:30:00 added to Canal Ruleta
Program Call the Midwife with id 902093079 added to DB.
Showtime starting on 2018-02-26 02:40:00 added to Call the Midwife
Showtime starting on 2018-02-26 03:15:00 added to Claire Byrne Live
Showtime starting on 2018-02-26 03:30:00 added to Minutos 

Showtime starting on 2018-02-27 03:55:00 added to Minutos musicales
Showtime starting on 2018-02-27 04:10:00 added to Shortland Street
Showtime starting on 2018-02-27 04:35:00 added to Nationwide
Showtime starting on 2018-02-27 05:00:00 added to Dr. Phil
Showtime starting on 2018-02-27 05:00:00 added to Minutos musicales
Showtime starting on 2018-02-27 05:15:00 added to Navy: investigación criminal
Showtime starting on 2018-02-27 05:45:00 added to Petits secrets entre voisins
Showtime starting on 2018-02-27 05:45:00 added to Euronews
Showtime starting on 2018-02-27 06:00:00 added to Euronews
Showtime starting on 2018-02-27 06:10:00 added to Petits secrets entre voisins
Showtime starting on 2018-02-27 06:20:00 added to Teleshopping
Showtime starting on 2018-02-27 06:30:00 added to ¿Quién vive ahí?
Showtime starting on 2018-02-27 06:40:00 added to Petits secrets entre voisins
Showtime starting on 2018-02-27 07:05:00 added to Petits secrets entre voisins
Program My Family with id 12048173

Program Dr. Phil with id 4661853759 added to DB.
Showtime starting on 2018-02-28 10:25:00 added to Dr. Phil
Showtime starting on 2018-02-28 10:40:00 added to Au nom de la vérité
Showtime starting on 2018-02-28 11:10:00 added to Au nom de la vérité
Program Shortland Street with id 5837038309 added to DB.
Showtime starting on 2018-02-28 11:20:00 added to Shortland Street
Showtime starting on 2018-02-28 11:40:00 added to Au nom de la vérité
Program Leaders' Questions with id 12140764 added to DB.
Showtime starting on 2018-02-28 11:50:00 added to Leaders' Questions
Program Au nom de la vérité with id 5827250859 added to DB.
Showtime starting on 2018-02-28 12:10:00 added to Au nom de la vérité
Showtime starting on 2018-02-28 12:50:00 added to Petits plats en équilibre
Showtime starting on 2018-02-28 12:55:00 added to Urgences
Showtime starting on 2018-02-28 13:00:00 added to laSexta Noticias 1ª Edición
Showtime starting on 2018-02-28 13:00:00 added to RTÉ News: One O'Clock
Program Home And 

In [5]:
df.sample(3)

,channel_id,channel_name,channel_country,start_time,program_id,program_original_title,program_local_title,program_year,end_time
3986,69036687,RTE 1,IRL,2018-02-18 08:40:00,3986112242,Casualty,Casualty,2016.0,2018-02-18 10:40:00
685,69036687,RTE 1,IRL,2018-01-09 14:00:00,5644925644,Neighbours,Neighbours,n/a,2018-01-09 14:25:00
886,69036190,La Sexta,ESP,2018-01-12 00:05:00,4866635142,Dentro de,Dentro de,2017.0,2018-01-12 01:10:00
